# Before you use this template

This template is just a recommended template for project Report. It only considers the general type of research in our paper pool. Feel free to edit it to better fit your project. You will iteratively update the same notebook submission for your draft and the final submission. Please check the project rubriks to get a sense of what is expected in the template.

---

# FAQ and Attentions
* Copy and move this template to your Google Drive. Name your notebook by your team ID (upper-left corner). Don't eidt this original file.
* This template covers most questions we want to ask about your reproduction experiment. You don't need to exactly follow the template, however, you should address the questions. Please feel free to customize your report accordingly.
* any report must have run-able codes and necessary annotations (in text and code comments).
* The notebook is like a demo and only uses small-size data (a subset of original data or processed data), the entire runtime of the notebook including data reading, data process, model training, printing, figure plotting, etc,
must be within 8 min, otherwise, you may get penalty on the grade.
  * If the raw dataset is too large to be loaded  you can select a subset of data and pre-process the data, then, upload the subset or processed data to Google Drive and load them in this notebook.
  * If the whole training is too long to run, you can only set the number of training epoch to a small number, e.g., 3, just show that the training is runable.
  * For results model validation, you can train the model outside this notebook in advance, then, load pretrained model and use it for validation (display the figures, print the metrics).
* The post-process is important! For post-process of the results,please use plots/figures. The code to summarize results and plot figures may be tedious, however, it won't be waste of time since these figures can be used for presentation. While plotting in code, the figures should have titles or captions if necessary (e.g., title your figure with "Figure 1. xxxx")
* There is not page limit to your notebook report, you can also use separate notebooks for the report, just make sure your grader can access and run/test them.
* If you use outside resources, please refer them (in any formats). Include the links to the resources if necessary.

# Mount Notebook to Google Drive
Upload the data, pretrianed model, figures, etc to your Google Drive, then mount this notebook to Google Drive. After that, you can access the resources freely.

Instruction: https://colab.research.google.com/notebooks/io.ipynb

Example: https://colab.research.google.com/drive/1srw_HFWQ2SMgmWIawucXfusGzrj1_U0q

Video: https://www.youtube.com/watch?v=zc8g8lGcwQU

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# Introduction
This is an introduction to your report, you should edit this text/mardown section to compose. In this text/markdown, you should introduce:

*   Background of the problem
  * what type of problem: disease/readmission/mortality prediction,  feature engineeing, data processing, etc
  * what is the importance/meaning of solving the problem
  * what is the difficulty of the problem
  * the state of the art methods and effectiveness.
*   Paper explanation
  * what did the paper propose
  * what is the innovations of the method
  * how well the proposed method work (in its own metrics)
  * what is the contribution to the reasearch regime (referring the Background above, how important the paper is to the problem).


In [2]:
# code comment is used as inline annotations for your coding

# Scope of Reproducibility:

List hypotheses from the paper you will test and the corresponding experiments you will run.


1.   Hypothesis 1: xxxxxxx
2.   Hypothesis 2: xxxxxxx

You can insert images in this notebook text, [see this link](https://stackoverflow.com/questions/50670920/how-to-insert-an-inline-image-in-google-colaboratory-from-google-drive) and example below:

![sample_image.png](https://drive.google.com/uc?export=view&id=1g2efvsRJDxTxKz-OY3loMhihrEUdBxbc)



You can also use code to display images, see the code below.

The images must be saved in Google Drive first.


In [3]:
# no code is required for this section
'''
if you want to use an image outside this notebook for explanaition,
you can upload it to your google drive and show it with OpenCV or matplotlib
'''
# mount this notebook to your google drive
# drive.mount('/content/gdrive')

# define dirs to workspace and data
# img_dir = '/content/gdrive/My Drive/Colab Notebooks/<path-to-your-image>'

# import cv2
# img = cv2.imread(img_dir)
# cv2.imshow("Title", img)


'\nif you want to use an image outside this notebook for explanaition,\nyou can upload it to your google drive and show it with OpenCV or matplotlib\n'

# Methodology

This methodology is the core of your project. It consists of run-able codes with necessary annotations to show the expeiment you executed for testing the hypotheses.

The methodology at least contains two subsections **data** and **model** in your experiment.

In [5]:
# import  packages you need
import numpy as np
# from google.colab import drive


##  Data
Data includes raw data (MIMIC III tables), descriptive statistics (our homework questions), and data processing (feature engineering).
  * Source of the data: where the data is collected from; if data is synthetic or self-generated, explain how. If possible, please provide a link to the raw datasets.
  * Statistics: include basic descriptive statistics of the dataset like size, cross validation split, label distribution, etc.
  * Data process: how do you munipulate the data, e.g., change the class labels, split the dataset to train/valid/test, refining the dataset.
  * Illustration: printing results, plotting figures for illustration.
  * You can upload your raw dataset to Google Drive and mount this Colab to the same directory. If your raw dataset is too large, you can upload the processed dataset and have a code to load the processed dataset.

# Preprocessing and loading CHAT data

In [ ]:
from typing import TypeVar
from collections import Mapping

K = TypeVar("K")
V = TypeVar("V")

class ReadOnlyDict(Mapping[K, V]):

    def __init__(self, data: dict[K, V]):
        self._data = data

    def __getitem__(self, key: K) -> V: 
        return self._data[key]

    def __len__(self) -> int:
        return len(self._data)

    def __iter__(self):
        return iter(self._data)

    def as_dict(self):
        return self._data.copy()


In [ ]:
import glob
import os
from datetime import datetime
from itertools import compress

import mne
import numpy as np
import pandas as pd
from mne import make_fixed_length_events

THRESHOLD = 3
NUM_WORKER = 1
SN = 3984  # STUDY NUMBER
FREQ = 128.0
EPOCH_LENGTH = 30.0
OUT_FOLDER = 'E:\\data_chat_baseline_30x128_test'

channels: list[str] = [
    'E1',  # 0
    'E2',  # 1
    'F3',  # 2
    'F4',  # 3
    'C3',  # 4
    'C4',  # 5
    'M1',  # 6
    'M2',  # 7
    'O1',  # 8
    'O2',  # 9
    'ECG1',  # 10
    'ECG3',  # 11

    'CANNULAFLOW',  # 12
    'AIRFLOW',  # 13
    'CHEST',  # 14
    'ABD',  # 15
    'SAO2',  # 16
    'CAP',  # 17
]

APNEA_EVENT_DICT = ReadOnlyDict({
    "Obstructive apnea": 2,
    "Central apnea": 2,
})

HYPOPNEA_EVENT_DICT = ReadOnlyDict({
    "Hypopnea": 1,
})

POS_EVENT_DICT = ReadOnlyDict({
    "Hypopnea": 1,
    "Obstructive apnea": 2,
    "Central apnea": 2,
})

NEG_EVENT_DICT = ReadOnlyDict({
    'Stage 1 sleep': 0,
    'Stage 2 sleep': 0,
    'Stage 3 sleep': 0,
    'REM sleep': 0,
})

WAKE_DICT = ReadOnlyDict({
    "Wake": 10
})

mne.set_log_file('log.txt', overwrite=False)


########################################## Annotation Modifier functions ##########################################

def change_duration(
        df,
        label_dict: ReadOnlyDict[str, int]=POS_EVENT_DICT,
        duration:float=EPOCH_LENGTH
    ):
    for key in label_dict:
        df.loc[df.description == key, 'duration'] = duration
    print("change duration!")
    return df


def load_study_chat(edf_path, annotation_path, annotation_func, preload=False, exclude=[], verbose='CRITICAL'):
    raw = mne.io.edf.edf.RawEDF(input_fname=edf_path, exclude=exclude, preload=preload, verbose=verbose)

    df = annotation_func(pd.read_csv(annotation_path, sep='\t'))
    annotations = mne.Annotations(df.onset, df.duration, df.description)  # ,orig_time=new_datetime)

    raw.set_annotations(annotations)

    raw.rename_channels({name: name.upper() for name in raw.info['ch_names']})

    return raw


def preprocess(path: str, annotation_modifier: str, out_dir: str):
    is_apnea_available, is_hypopnea_available = True, True
    raw = load_study_chat(
        path[0],
        path[1],
        annotation_modifier,
        verbose=True
    )
    ########################################   CHECK CRITERIA FOR SS   #################################################
    if not all([name in raw.ch_names for name in channels]):
        print([name in raw.ch_names for name in channels])
        print("study " + os.path.basename(path[0]) + " skipped since insufficient channels")
        return 0

    try:
        apnea_events, event_ids = mne.events_from_annotations(
            raw=raw,
            event_id=POS_EVENT_DICT,
            chunk_duration=1.0,
            verbose=None,
        )
    except ValueError:
        print("No Chunk found!")
        return 0
    ########################################   CHECK CRITERIA FOR SS   #################################################
    print(str(datetime.now().time().strftime("%H:%M:%S")) + ' --- Processing %s' % os.path.basename(path[0]))

    try:
        apnea_events, event_ids = mne.events_from_annotations(
            raw=raw,
            event_id=APNEA_EVENT_DICT,
            chunk_duration=1.0,
            verbose=None,
        )
    except ValueError:
        is_apnea_available = False

    try:
        hypopnea_events, event_ids = mne.events_from_annotations(
            raw=raw,
            event_id=HYPOPNEA_EVENT_DICT,
            chunk_duration=1.0,
            verbose=None,
        )
    except ValueError:
        is_hypopnea_available = False

    wake_events, event_ids = mne.events_from_annotations(raw, event_id=WAKE_DICT, chunk_duration=1.0, verbose=None)
    ####################################################################################################################
    sfreq = raw.info['sfreq']
    tmax = EPOCH_LENGTH - 1. / sfreq

    raw = raw.pick_channels(channels, ordered=True)
    fixed_events = make_fixed_length_events(raw, id=0, duration=EPOCH_LENGTH, overlap=0.)
    epochs = mne.Epochs(raw, fixed_events, event_id=[0], tmin=0, tmax=tmax, baseline=None, preload=True, proj=False, verbose=None)
    epochs.load_data()
    if sfreq != FREQ:
        epochs = epochs.resample(FREQ, npad='auto', n_jobs=8, verbose=None)
    data = epochs.get_data()
    ####################################################################################################################
    if is_apnea_available:
        apnea_events_set = set((apnea_events[:, 0] / sfreq).astype(int))
    if is_hypopnea_available:
        hypopnea_events_set = set((hypopnea_events[:, 0] / sfreq).astype(int))
    wake_events_set = set((wake_events[:, 0] / sfreq).astype(int))

    starts = (epochs.events[:, 0] / sfreq).astype(int)

    labels_apnea = []
    labels_hypopnea = []
    labels_not_awake = []
    total_apnea_event_second = 0
    total_hypopnea_event_second = 0

    for seq in range(data.shape[0]):
        epoch_set = set(range(starts[seq], starts[seq] + int(EPOCH_LENGTH)))
        if is_apnea_available:
            apnea_seconds = len(apnea_events_set.intersection(epoch_set))
            total_apnea_event_second += apnea_seconds
            labels_apnea.append(apnea_seconds)
        else:
            labels_apnea.append(0)

        if is_hypopnea_available:
            hypopnea_seconds = len(hypopnea_events_set.intersection(epoch_set))
            total_hypopnea_event_second += hypopnea_seconds
            labels_hypopnea.append(hypopnea_seconds)
        else:
            labels_hypopnea.append(0)

        labels_not_awake.append(len(wake_events_set.intersection(epoch_set)) == 0)
    ####################################################################################################################
    data = data[labels_not_awake, :, :]
    labels_apnea = list(compress(labels_apnea, labels_not_awake))
    labels_hypopnea = list(compress(labels_hypopnea, labels_not_awake))
    ####################################################################################################################

    new_data: np.ndarray = np.zeros_like(data)
    for i in range(data.shape[0]):

        new_data[i, 0, :] = data[i, 0, :] - data[i, 7, :]  # E1 - M2
        new_data[i, 1, :] = data[i, 1, :] - data[i, 6, :]  # E2 - M1

        new_data[i, 2, :] = data[i, 2, :] - data[i, 7, :]  # F3 - M2
        new_data[i, 3, :] = data[i, 3, :] - data[i, 6, :]  # F4 - M1
        new_data[i, 4, :] = data[i, 4, :] - data[i, 7, :]  # C3 - M2
        new_data[i, 5, :] = data[i, 5, :] - data[i, 6, :]  # C4 - M1
        new_data[i, 6, :] = data[i, 8, :] - data[i, 7, :]  # O1 - M2
        new_data[i, 7, :] = data[i, 9, :] - data[i, 6, :]  # O2 - M1

        new_data[i, 8, :] = data[i, 10,:] - data[i, 11,:]  # ECG

        new_data[i, 9, :] = data[i, 12, :]  # CANULAFLOW
        new_data[i, 10, :] = data[i, 13, :]  # AIRFLOW
        new_data[i, 11, :] = data[i, 14, :]  # CHEST
        new_data[i, 12, :] = data[i, 15, :]  # ABD
        new_data[i, 13, :] = data[i, 16, :]  # SAO2
        new_data[i, 14, :] = data[i, 17, :]  # CAP
    data = new_data[:, :15, :]
    ####################################################################################################################

    np.savez_compressed(
        file=(
            out_dir + '\\' + os.path.basename(path[0]) + "_" + 
            str(total_apnea_event_second) + "_" + 
            str(total_hypopnea_event_second)
        ),
        data=data,
        labels_apnea=labels_apnea,
        labels_hypopnea=labels_hypopnea,
    )

    return data.shape[0]


root = "./chat/polysomnography/edfs/baseline_test/"
for edf_file in glob.glob(root + "*.edf"):
    annot_file = edf_file.replace(".edf", "-nsrr.tsv")

    preprocess((edf_file, annot_file), lambda a: a, OUT_FOLDER)

In [ ]:
# dir and function to load raw data
raw_data_dir = '/content/gdrive/My Drive/Colab Notebooks/<path-to-raw-data>'

def load_raw_data(raw_data_dir):
  # implement this function to load raw data to dataframe/numpy array/tensor
  return None

raw_data = load_raw_data(raw_data_dir)

# calculate statistics
def calculate_stats(raw_data):
  # implement this function to calculate the statistics
  # it is encouraged to print out the results
  return None

# process raw data
def process_data(raw_data):
    # implement this function to process the data as you need
  return None

processed_data = process_data(raw_data)

''' you can load the processed data directly
processed_data_dir = '/content/gdrive/My Drive/Colab Notebooks/<path-to-raw-data>'
def load_processed_data(raw_data_dir):
  pass

'''

##   Model
The model includes the model definitation which usually is a class, model training, and other necessary parts.
  * Model architecture: layer number/size/type, activation function, etc
  * Training objectives: loss function, optimizer, weight of each loss term, etc
  * Others: whether the model is pretrained, Monte Carlo simulation for uncertainty analysis, etc
  * The code of model should have classes of the model, functions of model training, model validation, etc.
  * If your model training is done outside of this notebook, please upload the trained model here and develop a function to load and test it.

In [ ]:
class my_model():
  # use this class to define your model
  pass

model = my_model()
loss_func = None
optimizer = None

def train_model_one_iter(model, loss_func, optimizer):
  pass

num_epoch = 10
# model training loop: it is better to print the training/validation losses during the training
for i in range(num_epoch):
  train_model_one_iter(model, loss_func, optimizer)
  train_loss, valid_loss = None, None
  print("Train Loss: %.2f, Validation Loss: %.2f" % (train_loss, valid_loss))


# Results
In this section, you should finish training your model training or loading your trained model. That is a great experiment! You should share the results with others with necessary metrics and figures.

Please test and report results for all experiments that you run with:

*   specific numbers (accuracy, AUC, RMSE, etc)
*   figures (loss shrinkage, outputs from GAN, annotation or label of sample pictures, etc)


In [ ]:
# metrics to evaluate my model

# plot figures to better show the results

# it is better to save the numbers and figures for your presentation.

## Model comparison

In [ ]:
# compare you model with others
# you don't need to re-run all other experiments, instead, you can directly refer the metrics/numbers in the paper

# Discussion

In this section,you should discuss your work and make future plan. The discussion should address the following questions:
  * Make assessment that the paper is reproducible or not.
  * Explain why it is not reproducible if your results are kind negative.
  * Describe “What was easy” and “What was difficult” during the reproduction.
  * Make suggestions to the author or other reproducers on how to improve the reproducibility.
  * What will you do in next phase.



In [ ]:
# no code is required for this section
'''
if you want to use an image outside this notebook for explanaition,
you can read and plot it here like the Scope of Reproducibility
'''

# References

1.   Sun, J, [paper title], [journal title], [year], [volume]:[issue], doi: [doi link to paper]



# Feel free to add new sections